In [49]:
import re
import glob
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)



In [156]:

filename=glob.glob("*baux*.xlsx")[0]
df_fichier_baux = pd.read_excel(filename)

col_fin_bail =  'Fin Bail'
col_type_bail =  'Type Bail'
col_numero_bail =  'Numero Mandat'

df_fichier_baux=df_fichier_baux.rename(columns={'20/05/2022.1': col_fin_bail})
df_fichier_baux=df_fichier_baux.rename(columns={'Unnamed: 17': col_type_bail})
df_fichier_baux=df_fichier_baux.rename(columns={'Unnamed: 0': col_numero_bail})


for col in df_fichier_baux.columns:
    print(col)
    break
print("***")

df_fichier_baux = df_fichier_baux[df_fichier_baux[col_fin_bail].notna()]

# df_new_df=df_fichier_baux[col_fin_bail].astype(str)+df_fichier_baux[col_type_bail].astype(str)
df_new_df=pd.concat([df_fichier_baux[col_numero_bail],df_fichier_baux[col_type_bail],df_fichier_baux[col_fin_bail]], axis=1)
print(df_new_df.head())

Numero Mandat
***
   Numero Mandat Type Bail    Fin Bail
5   00046-001-06         B  04/11/2022
7   00051-001-03         B  31/10/2022
12  00113-001-02         4  31/10/2007
15  00134-002-03         0  18/10/2022
17  00137-001-02         4  02/12/2018


In [181]:
# 0 : 8 mois avant date de fin
# 3 : 8mois 
# 4 : à la date de fin + (date de fin + 3 ans - 6 mois)
# B : 8 
# C : 8
# G : 8

from datetime import datetime, timedelta, date

six_months = timedelta(6*365/12)
height_months = timedelta(8*365/12)
three_years = timedelta(3*365)
today_date = datetime.now()

for indices, row in df_new_df.iterrows():
    date = df_new_df.at[indices,"Fin Bail"]
    date_to_time_obj = datetime.strptime(date, '%d/%m/%Y')
    type_bail = str(df_new_df.at[indices,"Type Bail"])
    match type_bail :
        case '4':
            if today_date == date_to_time_obj or today_date >= date_to_time_obj+three_years-six_months :
                print(type_bail,date_to_time_obj,"expired")
                pass
            else:
                print(type_bail,date_to_time_obj,"not expired")
        case _:
            if today_date >= date_to_time_obj-height_months:
                print(type_bail,date_to_time_obj,"expired")
                pass


B 2022-11-04 00:00:00 expired
B 2022-10-31 00:00:00 expired
4 2007-10-31 00:00:00 expired
0 2022-10-18 00:00:00 expired
4 2018-12-02 00:00:00 expired
C 2022-12-14 00:00:00 expired
0 2022-11-30 00:00:00 expired
4 2019-12-13 00:00:00 not expired
0 2022-12-16 00:00:00 expired
B 2022-11-03 00:00:00 expired
C 2022-12-13 00:00:00 expired
4 2019-06-30 00:00:00 expired
4 2020-12-31 00:00:00 not expired
4 2022-04-04 00:00:00 not expired
G 2022-08-31 00:00:00 expired
G 2022-07-31 00:00:00 expired
0 2022-10-01 00:00:00 expired
B 2022-11-04 00:00:00 expired
0 2022-12-03 00:00:00 expired
4 2011-01-31 00:00:00 expired
4 2011-01-31 00:00:00 expired
4 2020-01-30 00:00:00 not expired
4 2022-03-14 00:00:00 not expired
4 2018-12-31 00:00:00 expired
4 2021-05-31 00:00:00 not expired
4 2021-05-31 00:00:00 not expired
4 2021-05-31 00:00:00 not expired
4 2021-05-31 00:00:00 not expired
4 2022-06-30 00:00:00 not expired
4 2022-07-16 00:00:00 not expired
4 2019-10-31 00:00:00 expired
4 2021-07-31 00:00:00 not 

In [158]:
from datetime import datetime, timedelta, date

# Numpy vectorization
list=(df_new_df['Fin Bail'].to_numpy()).tolist()
print(list)

if 0:
  date_time_str = '18/09/19'

  date_time_obj = datetime.strptime(date_time_str, '%d/%m/%y')

  previous_date = datetime.now() - timedelta(days=1)
  current_date = datetime.now()
  print(previous_date,current_date)
  print(previous_date > current_date)


  def add_has_expired(df):
    return df.assign(alert_mail=['yes' if datetime.strptime(x, "%d/%m/%Y") <= current_date else 'no' for x in df['Fin Bail']])


['04/11/2022', '31/10/2022', '31/10/2007', '18/10/2022', '02/12/2018', '14/12/2022', '30/11/2022', '13/12/2019', '16/12/2022', '03/11/2022', '13/12/2022', '30/06/2019', '31/12/2020', '04/04/2022', '31/08/2022', '31/07/2022', '01/10/2022', '04/11/2022', '03/12/2022', '31/01/2011', '31/01/2011', '30/01/2020', '14/03/2022', '31/12/2018', '31/05/2021', '31/05/2021', '31/05/2021', '31/05/2021', '30/06/2022', '16/07/2022', '31/10/2019', '31/07/2021', '30/09/2020', '31/10/2018', '30/09/2018', '30/09/2018', '14/07/2020', '30/06/2012', '30/06/2018', '19/02/2021']


In [157]:
import smtplib, ssl
from email.message import EmailMessage

msg = EmailMessage()
msg.set_content("The body of the email is here")
msg["Subject"] = "Rappel automatique - Baux bientôt arrivés à expiration"
msg["From"] = "atoualbi@sve-universite.com"
msg["To"] = ["atoualbi@sve-universite.com"] 

PASSWORD = "Abdelsve5"
SMTP = "smtp-mail.outlook.com"

context=ssl.create_default_context()

with smtplib.SMTP(SMTP, port=587) as smtp:
    smtp.starttls(context=context)
    smtp.login(msg["From"], PASSWORD)
    # smtp.send_message(msg)